In [1]:
import numpy as np
import os
from finn.core.datatype import DataType
from final_011903.output_final.driver.driver_base import FINNExampleOverlay
from tqdm import tqdm
# import torch

In [2]:
build_dir = "final_011903"

In [3]:
bitfile = f"./{build_dir}/output_final/bitfile/finn-accel.bit"
platform = "zynq-iodma"
io_shape_dict = {
    # FINN DataType for input and output tensors
    "idt" : [DataType['INT8']],
    "odt" : [DataType['INT8']],
    # shapes for input and output tensors (NHWC layout)
    "ishape_normal" : [(1, 1, 128, 1)],
    "oshape_normal" : [(1, 8)],
    # folded / packed shapes below depend on idt/odt and input/output
    # PE/SIMD parallelization settings -- these are calculated by the
    # FINN compiler.
    "ishape_folded" : [(1, 1, 128, 1, 1)],
    "oshape_folded" : [(1, 8, 1)],
    "ishape_packed" : [(1, 1, 128, 1, 1)],
    "oshape_packed" : [(1, 8, 1)],
    "input_dma_name" : ['idma0'],
    "output_dma_name" : ['odma0'],
    "number_of_external_weights": 0,
    "num_inputs" : 1,
    "num_outputs" : 1,
}
batch_size = 1
runtime_weight_dir = f"./{build_dir}/runtime_weights/"


verif_model_filename = "./{build_dir}/brevitas07-IEEE-verification.onnx"

In [4]:
# inputfile_0 = "./val_data/0/101_100722.npy"
# inputfile_1 = "./val_data/1/105_198328.npy"
# inputfile_2 = "./val_data/2/109_100722.npy"
# inputfile_3 = "./val_data/3/107_101575.npy"
# inputfile_4 = "./val_data/4/118_101077.npy"
# inputfile_5 = "./val_data/5/101_532014.npy"
# inputfile_6 = "./val_data/6/207_597800.npy"
# inputfile_7 = "./val_data/7/207_15258.npy"
outputfile = "output.npy"

In [5]:
import os
file_list = os.listdir("./val_data/6")
print(file_list)

['207_619539.npy', '207_606449.npy', '207_621387.npy', '207_611866.npy', '207_615728.npy', '207_597800.npy', '207_609371.npy', '210_634129.npy', '207_606126.npy', '207_612171.npy', '207_609682.npy']


In [6]:
%%time
accel = FINNExampleOverlay(
        bitfile_name = bitfile, platform = platform,
        io_shape_dict = io_shape_dict, batch_size = batch_size,
        runtime_weight_dir = runtime_weight_dir
    )

CPU times: user 444 ms, sys: 100 ms, total: 544 ms
Wall time: 626 ms


In [8]:
%%time
nowIndex = 6
total_corr = 0
total_len = 0
for nowIndex in range(0,8):
    nowDir = f"./val_data/{nowIndex}/"
    file_list = os.listdir(nowDir)
    correct = 0
    for file in file_list:
    #     print(file)
        ibuf_normal = np.load(nowDir+file).astype("float32")

        mean_val = ibuf_normal.mean()
        ibuf_normal = ibuf_normal-mean_val
        range_val = ibuf_normal.max() if ibuf_normal.max() > -ibuf_normal.min() else -ibuf_normal.min()
        range_val = range_val / 120
        ibuf_normal = (ibuf_normal/range_val)
        # print(mean_val, range_val)
        # plt.plot(img)
        # plt.show()
        # plt.close()
        # print(img)

        ibuf_normal = ibuf_normal.reshape(1,1,128,1).astype(np.int8)
    #     print(ibuf_normal.max(),ibuf_normal.min())
    #     print(ibuf_normal.shape)

        obuf_normal = accel.execute(ibuf_normal)
#         print(obuf_normal.argmax() , nowIndex)
        if obuf_normal.argmax() == nowIndex:
            correct += 1
    print(f"nowIndex:{nowIndex},acc:{correct/len(file_list)}, ({correct}/{len(file_list)})")
    total_corr += correct
    total_len += len(file_list)
print(f"TOTAL acc:{total_corr/total_len}, ({total_corr}/{total_len})")

nowIndex:0,acc:0.9220917822838848, (6048/6559)
nowIndex:1,acc:0.7112375533428165, (500/703)
nowIndex:2,acc:0.8228004956629492, (664/807)
nowIndex:3,acc:0.6077348066298343, (220/362)
nowIndex:4,acc:0.9284467713787086, (532/573)
nowIndex:5,acc:0.6, (150/250)
nowIndex:6,acc:0.8181818181818182, (9/11)
nowIndex:7,acc:0.10638297872340426, (5/47)
TOTAL acc:0.872852233676976, (8128/9312)
CPU times: user 1min 7s, sys: 1.68 s, total: 1min 8s
Wall time: 1min 9s


In [11]:
%%time
nowIndex = 6
total_corr = 0
total_len = 0
for nowIndex in range(0,8):
    nowDir = f"./val_data/{nowIndex}/"
    file_list = os.listdir(nowDir)
    correct = 0
    for file in file_list:
    #     print(file)
        ibuf_normal = np.load(nowDir+file).astype("float32")

        mean_val = ibuf_normal.mean()
        ibuf_normal = ibuf_normal-mean_val
        range_val = ibuf_normal.max() if ibuf_normal.max() > -ibuf_normal.min() else -ibuf_normal.min()
        range_val = range_val / 120
        ibuf_normal = (ibuf_normal/range_val)
        # print(mean_val, range_val)
        # plt.plot(img)
        # plt.show()
        # plt.close()
        # print(img)

        ibuf_normal = ibuf_normal.reshape(1,1,128,1).astype(np.int8)
    #     print(ibuf_normal.max(),ibuf_normal.min())
    #     print(ibuf_normal.shape)

        obuf_normal = ibuf_normal
#         print(obuf_normal)
        if obuf_normal.argmax() == nowIndex:
            correct += 1
    print(f"nowIndex:{nowIndex},acc:{correct/len(file_list)}, ({correct}/{len(file_list)})")
    total_corr += correct
    total_len += len(file_list)
print(f"TOTAL acc:{total_corr/total_len}, ({total_corr}/{total_len})")

nowIndex:0,acc:0.04329928342735173, (284/6559)
nowIndex:1,acc:0.002844950213371266, (2/703)
nowIndex:2,acc:0.0, (0/807)
nowIndex:3,acc:0.008287292817679558, (3/362)
nowIndex:4,acc:0.0, (0/573)
nowIndex:5,acc:0.0, (0/250)
nowIndex:6,acc:0.0, (0/11)
nowIndex:7,acc:0.02127659574468085, (1/47)
TOTAL acc:0.031142611683848798, (290/9312)
CPU times: user 43.9 s, sys: 926 ms, total: 44.8 s
Wall time: 44.8 s


In [12]:
try:
    os.remove(f"nw_metrics_{build_dir}.txt")
except FileNotFoundError:
    pass

res = accel.throughput_test()
file = open(f"nw_metrics_{build_dir}.txt", "w")
file.write(str(res))
file.close()
print("Results written to nw_metrics.txt")

Results written to nw_metrics.txt
